In [1]:
from fastai2.text.all import *

In [2]:
path = Path.home()/'.fastai/data/nlp-getting-started'

In [3]:
df_train = pd.read_csv(path/'train.csv')
df_test = pd.read_csv(path/'test.csv')
df = pd.concat([df_train, df_test], axis=0, sort=True)
df.head(2)

,id,keyword,location,target,text
0,1,NaN,NaN,1.0,Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all
1,4,NaN,NaN,1.0,Forest fire near La Ronge Sask. Canada


# Approach A

We will follow https://github.com/fastai/fastbook/blob/master/10_nlp.ipynb 

Older reference - https://github.com/muellerzr/Practical-Deep-Learning-for-Coders-2.0/blob/master/Text%20Notebooks%20(old)/09a_IMDB_Sample.ipynb (on handling dataframes)

In [4]:
text_transform_block = TextBlock.from_df(text_cols=['text'], is_lm=True)

# DataBlock is a "Generic container to quickly build `Datasets` and `DataLoaders`"

lm_dl = DataBlock(
    blocks=text_transform_block,
    get_x=attrgetter('text'),
    splitter=RandomSplitter()
).dataloaders(df, bs=128, seq_len=72)

lm_dl.show_batch(max_n=2)

,text,text_
0,xxbos xxunk xxmaj now xxunk it up because that 's all you 're bloody getting out of me by means of xxunk . xxbos xxmaj to xxmaj all xxmaj the meat - loving xxmaj feminists xxmaj of xxmaj the xxmaj world xxmaj riot xxmaj grill xxmaj has xxmaj arrived : xxmaj pop quiz ! xxmaj which do you prefer : xxunk … http : / / t.co / xxunk # art xxbos,xxunk xxmaj now xxunk it up because that 's all you 're bloody getting out of me by means of xxunk . xxbos xxmaj to xxmaj all xxmaj the meat - loving xxmaj feminists xxmaj of xxmaj the xxmaj world xxmaj riot xxmaj grill xxmaj has xxmaj arrived : xxmaj pop quiz ! xxmaj which do you prefer : xxunk … http : / / t.co / xxunk # art xxbos xxup
1,heat xxunk _ ? ? û _ http : / / t.co / xxunk http : / / t.co / xxunk xxbos xxunk : xxrep 6 ? xxmaj no xxmaj xxunk xxmaj needed ? ? . xxmaj xxunk … http : / / t.co / xxunk nigga xxmaj stacey body bagging niggas ! xxrep 4 ? xxbos xxmaj mad xxmaj river xxmaj complex fires have burned xxunk acres 8 % contained :,xxunk _ ? ? û _ http : / / t.co / xxunk http : / / t.co / xxunk xxbos xxunk : xxrep 6 ? xxmaj no xxmaj xxunk xxmaj needed ? ? . xxmaj xxunk … http : / / t.co / xxunk nigga xxmaj stacey body bagging niggas ! xxrep 4 ? xxbos xxmaj mad xxmaj river xxmaj complex fires have burned xxunk acres 8 % contained : xxmaj


In [5]:
print('vocab len: ', len(lm_dl.vocab))

vocab len:  5832


In [6]:
print(text_transform_block.type_tfms)
print(text_transform_block.item_tfms)
print(text_transform_block.batch_tfms)

(#2) [Tokenizer: (str,object) -> encodes
(Path,object) -> encodes (object,object) -> decodes,Numericalize: (object,object) -> encodes (object,object) -> decodes]
(#1) [<class 'fastai2.data.transforms.ToTensor'>]
(#0) []


In [7]:
learn = language_model_learner(lm_dl, AWD_LSTM, metrics=Perplexity())
learn.fit_one_cycle(8, 2e-2, moms=(0.8, 0.7, 0.8))
learn.save('epoch_8')

epoch,train_loss,valid_loss,perplexity,time
0,5.381324,4.235898,69.123756,00:18
1,4.545297,3.513235,33.556652,00:18
2,4.039380,3.338115,28.165970,00:18
3,3.701136,3.259733,26.042595,00:18
4,3.476639,3.227166,25.208126,00:17
5,3.293758,3.206393,24.689857,00:18
6,3.168956,3.198973,24.507347,00:18
7,3.087549,3.197891,24.480833,00:18


In [8]:
learn.unfreeze()
learn.fit_one_cycle(4, 2e-3)
learn.save_encoder('finetuned')

epoch,train_loss,valid_loss,perplexity,time
0,2.927999,3.144534,23.208849,00:20
1,2.805745,3.085158,21.870914,00:20
2,2.697026,3.075958,21.670628,00:20
3,2.629881,3.083815,21.841566,00:20


## training for classification

In [83]:
clf_tranform_blocks = [TextBlock.from_df(text_cols=['text'], vocab=lm_dl.vocab), CategoryBlock]

print(clf_tranform_blocks[0].type_tfms)
print(clf_tranform_blocks[0].item_tfms)
print(clf_tranform_blocks[0].batch_tfms)

(#2) [Tokenizer: (str,object) -> encodes
(Path,object) -> encodes (object,object) -> decodes,Numericalize: (object,object) -> encodes (object,object) -> decodes]
(#1) [<class 'fastai2.data.transforms.ToTensor'>]
(#0) []


In [148]:
clf_dl = DataBlock(
    blocks=clf_tranform_blocks,
    get_x=attrgetter('text'),
    get_y=attrgetter('target'),
    splitter=RandomSplitter(0.1)
).dataloaders(df_train, verbose=True)

Setting up after_item: Pipeline: ToTensor
Setting up before_batch: Pipeline: partial
Setting up after_batch: Pipeline: 


In [89]:
clf_dl.show_batch(max_n=3)

,text,category
0,xxbos _ \n▁ xxrep 5 ? xxup retweet \n▁ xxrep 7 ? \n▁ xxrep 5 ? xxup follow xxup all xxup who xxup rt \n▁ xxrep 7 ? \n▁ xxrep 5 ? xxup xxunk \n▁ xxrep 7 ? \n▁ xxrep 5 ? xxup gain xxup with \n▁ xxrep 7 ? \n▁ xxrep 5 ? xxup follow ? xxunk # xxup xxunk \n▁ # xxup ty,0
1,xxbos . : . : . : . : . : . : . : . : . : . : . : . : . : . : . : . : . : . : . : . : . : xxup rt xxunk : # xxunk \n\n xxmaj indian xxmaj army xxunk _ http : / / t.co / xxunk g,0
2,xxbos xxup info xxup s. xxup wnd : xxunk / 6 . xxup xxunk : xxup xxunk xxup xxunk . xxup exp xxup inst xxup apch . xxup rwy 05 . xxup curfew xxup in xxup oper xxup until 2030 xxup z. xxup taxiways xxup foxtrot 5 & & xxup foxtrot 6 xxup navbl . xxup tmp : 10 .,0


In [90]:
learn = text_classifier_learner(clf_dl, AWD_LSTM, path=path, metrics=accuracy)
learn = learn.load_encoder('finetuned')

learn.fit_one_cycle(2, 2e-2, moms=(0.8, 0.7, 0.8))

epoch,train_loss,valid_loss,accuracy,time
0,0.668523,0.464901,0.785808,00:05
1,0.625037,0.450577,0.791064,00:05


In [91]:
learn.freeze_to(-2)
learn.fit_one_cycle(1, slice(1e-2/(2.6**4),1e-2))

epoch,train_loss,valid_loss,accuracy,time
0,0.535157,0.435110,0.802891,00:05


In [92]:
learn.freeze_to(-3)
learn.fit_one_cycle(1, slice(5e-3/(2.6**4),5e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.438039,0.446235,0.809461,00:07


In [93]:
learn.unfreeze()
learn.fit_one_cycle(2, slice(1e-3/(2.6**4),1e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.401646,0.453233,0.808147,00:08
1,0.384281,0.465995,0.822602,00:08


In [196]:
learn.export('tuned_classifier.pkl')

In [4]:
learn = load_learner('/home/achinta/.fastai/data/nlp-getting-started/tuned_classifier.pkl')

In [5]:
test_dl = learn.dls.test_dl(df_test)
# b = test_dl.one_batch()
# inp, preds, x, dec_preds = learn.get_preds(dl=test_dl, with_input=True, with_decoded=True)


In [179]:
# _,_,preds = learn.get_preds(dl=[b], with_decoded=True)
# accuracy_score(preds, df_train['target'])

0.5024300538552476

In [141]:
# inp, preds, x, dec_preds = learn.get_preds(0)
# idxs = []
# yhats = []
# ys = []
# for idx, pred in enumerate(dec_preds):
#     idxs.append(idx)
#     yhats.append(int(pred))  
#     ys.append(test_dl.items.iloc[idx].target)


In [180]:
# print(idxs[:20])
# print(yhats[:20])
# print(ys[:20])

In [6]:
# from tqdm import tqdm_notebook
ids = []
preds = []
targets = []

for idx in range(df_test.shape[0]):
    ids.append(test_dl.items.iloc[idx].id)
#     preds.append(test_dl.items.iloc[idx].target)
    preds.append(int(learn.predict(test_dl.items.iloc[idx])[1]))
    if idx%100 == 0:
        print(f'processed {idx}/{df_test.shape[0]}')

In [ ]:
output = pd.concat([pd.Series(ids), pd.Series(preds)], axis=1 )
output.columns=['id', 'target']
output.to_csv('output.csv', index=False)

In [22]:
!kaggle competitions submit -c nlp-getting-started -f output.csv -m 'fine tuning language model 2'

100%|##########| 22.2k/22.2k [00:10<00:00, 2.19kB/s]
Successfully submitted to Real or Not? NLP with Disaster Tweets

This gave a leaderboard score of 0.52862 (3287/3333)

In [54]:
test_dl.show_batch(max_n=2)

,text
0,xxbos xxup oh xxup my xxup xxunk xxup i m xxup at xxup my xxup xxunk xxup house xxup and xxup this xxup post xxup it xxup was xxup on xxup her xxup counter xxup and i xxup screamed xxup bc i xxup thought xxup it xxup said xxup chris xxup xxunk http : / / t.co / xxunk
1,xxbos xxmaj we d xxmaj aug 8 ! # xxmaj mayhem @ xxmaj xxunk ! 19 + xxmaj event ! xxmaj july & & xxmaj aug xxmaj xxunk xxmaj free ! xxmaj tix - > http : / / t.co / xxunk xxmaj xxunk ( xxunk ) xxunk - xxunk http : / / t.co / xxunk


In [56]:
df_test.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, stay safe everyone."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are fleeing across the street, I cannot save them all"
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [113]:
learn.dls[0]